In [4]:
import xgboost as xgb
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedKFold

In [5]:
model_xgb_baseline = xgb.Booster()
model_xgb_baseline.load_model("xgboost_traffic_LR1.json")

## Test on Kaggle

### Prepare test set

In [6]:
print("Reading Datafile...")
test_set = pd.read_csv('C:/Users/Admin/Documents/GitHub/CZ4041-NYC-Taxi-Fare/linear for traffic/testset_SGDR_traffic.csv')
print("Success!")
test_key_series = test_set['key'].squeeze()

Reading Datafile...
Success!


In [7]:
print("Removing extra columns...")
del test_set['key']
print("Test set is ready!")

Removing extra columns...
Test set is ready!


In [8]:
test_set

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,day,weekend,holiday,peak_hour,hotspot,dist,0
0,-73.973320,40.763805,-73.981430,40.743835,1,1,0,0,0,0,2.323260,0.083271
1,-73.986862,40.719383,-73.998886,40.739201,1,1,0,0,0,0,2.425353,0.083271
2,-73.982524,40.751260,-73.979654,40.746139,1,5,1,0,0,0,0.618628,0.399741
3,-73.981160,40.767807,-73.990448,40.751635,1,5,1,0,0,0,1.961033,0.399741
4,-73.966046,40.789775,-73.988565,40.744427,1,5,1,0,0,1,5.387301,0.699533
...,...,...,...,...,...,...,...,...,...,...,...,...
9909,-73.968124,40.796997,-73.955643,40.780388,6,6,1,0,0,0,2.124874,0.399741
9910,-73.945511,40.803600,-73.960213,40.776371,6,0,0,0,1,0,3.270969,0.383009
9911,-73.991600,40.726608,-73.789742,40.647011,6,6,1,0,0,0,19.183941,0.399741
9912,-73.985573,40.735432,-73.939178,40.801731,6,5,1,0,0,0,8.343486,0.399741


In [9]:
y_pred = model_xgb_baseline.predict(xgb.DMatrix(test_set), ntree_limit=model_xgb_baseline.best_ntree_limit)
y_pred

C:\Users\Admin\anaconda3\envs\ml\lib\site-packages\xgboost\core.py:90: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


array([ 8.627579 ,  8.88727  ,  4.908921 , ..., 48.083176 , 19.281116 ,
        6.3029046], dtype=float32)

In [10]:
y_pred = y_pred.flatten()

In [11]:
new_y_pred= []
for a in y_pred: 
    new_value = float("%.2f" % a)
    new_y_pred.append(new_value)
y_pred_series = pd.Series(new_y_pred)
df = pd.DataFrame({'key': test_key_series,'fare_amount': new_y_pred})
df

,key,fare_amount
0,2015-01-27 13:08:24.0000002,8.63
1,2015-01-27 13:08:24.0000003,8.89
2,2011-10-08 11:53:44.0000002,4.91
3,2012-12-01 21:12:12.0000002,7.90
4,2012-12-01 21:12:12.0000003,14.34
...,...,...
9909,2015-05-10 12:37:51.0000002,7.59
9910,2015-01-12 17:05:51.0000001,10.62
9911,2015-04-19 20:44:15.0000001,48.08
9912,2015-01-31 01:05:19.0000005,19.28


In [12]:
df.to_csv('XGBPrediction_Traffic_LR1.csv',index=False )

## Test on Validation Set

In [15]:
checkpoint5 = pd.read_csv('C:/Users/Admin/Documents/GitHub/CZ4041-NYC-Taxi-Fare/checkpoint5_SGDR_traffic.csv')
checkpoint5

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,time,day,weekend,holiday,peak_hour,hotspot,dist,0
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,17:26:21,0,0,0,1,0,1.030764,0.383009
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,16:52:16,1,0,0,1,0,8.450134,0.383009
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,00:35:00,3,0,0,0,1,1.389525,0.383063
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,04:30:42,5,1,0,0,1,2.799270,0.699533
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,07:51:00,1,0,0,1,0,1.999157,0.383009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54125939,14.0,-74.005272,40.740027,-73.963280,40.762555,1,03:28:00,5,1,0,0,0,4.334397,0.399741
54125940,4.2,-73.957784,40.765530,-73.951640,40.773959,1,20:46:20,1,0,0,0,0,1.070590,0.083271
54125941,14.1,-73.970505,40.752325,-73.960537,40.797342,1,22:04:24,5,1,0,0,0,5.075548,0.399741
54125942,28.9,-73.980901,40.764629,-73.870605,40.773963,1,05:57:51,2,0,0,0,0,9.346157,0.083271


In [16]:
labels = checkpoint5[['fare_amount']]
labels

,fare_amount
0,4.5
1,16.9
2,5.7
3,7.7
4,5.3
...,...
54125939,14.0
54125940,4.2
54125941,14.1
54125942,28.9


In [17]:
del checkpoint5['time']
del checkpoint5['fare_amount']
checkpoint5

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,day,weekend,holiday,peak_hour,hotspot,dist,0
0,-73.844311,40.721319,-73.841610,40.712278,1,0,0,0,1,0,1.030764,0.383009
1,-74.016048,40.711303,-73.979268,40.782004,1,1,0,0,1,0,8.450134,0.383009
2,-73.982738,40.761270,-73.991242,40.750562,2,3,0,0,0,1,1.389525,0.383063
3,-73.987130,40.733143,-73.991567,40.758092,1,5,1,0,0,1,2.799270,0.699533
4,-73.968095,40.768008,-73.956655,40.783762,1,1,0,0,1,0,1.999157,0.383009
...,...,...,...,...,...,...,...,...,...,...,...,...
54125939,-74.005272,40.740027,-73.963280,40.762555,1,5,1,0,0,0,4.334397,0.399741
54125940,-73.957784,40.765530,-73.951640,40.773959,1,1,0,0,0,0,1.070590,0.083271
54125941,-73.970505,40.752325,-73.960537,40.797342,1,5,1,0,0,0,5.075548,0.399741
54125942,-73.980901,40.764629,-73.870605,40.773963,1,2,0,0,0,0,9.346157,0.083271


In [18]:
print("splitting...")
#splitting data into train test 
x_train, x_test, y_train, y_test = train_test_split(checkpoint5, labels, test_size=0.2, random_state=42)

splitting...


In [19]:
y_pred = model_xgb_baseline.predict(xgb.DMatrix(x_test), ntree_limit=model_xgb_baseline.best_ntree_limit)
y_pred
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

C:\Users\Admin\anaconda3\envs\ml\lib\site-packages\xgboost\core.py:90: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


RMSE: 4.444704
